In [1]:
import matplotlib.pyplot as plt
import contextily as ctx
import geopandas as gpd
from pathlib import Path
import numpy as np
import pandas as pd
from folium import plugins
import folium



PROJECT_ROOT = Path().absolute().parent
DATA_DIR = PROJECT_ROOT / 'data'
RAW_DATA_DIR = DATA_DIR / 'raw'
PROCESSED_DATA_DIR = DATA_DIR / 'processed'
TEMP_DATA_DIR = DATA_DIR / 'temp'


# # Load environment variables
# load_dotenv()
# api_key = os.getenv('GOOGLE_MAPS_API_KEY')




In [10]:
def load_all_geodata(data_dir: Path):
    # Load station-related data
    station_buffers_gdf = gpd.read_file(data_dir / 'station_buffers.gpkg')
    station_loc_gdf = gpd.read_file(data_dir / 'station_loc.gpkg')
    
    # Load enumeration area layers
    grid_cells_gdf = gpd.read_file(data_dir / 'enumeration_area_data.gpkg', layer='grid_cells')
    centroids_gdf = gpd.read_file(data_dir / 'enumeration_area_data.gpkg', layer='centroids')
    road_points_gdf = gpd.read_file(data_dir / 'enumeration_area_data.gpkg', layer='road_points')
    village_points_gdf = gpd.read_file(data_dir / 'enumeration_area_data.gpkg', layer='village_points')
    
    # Ensure all data is in WGS84 for Folium
    station_buffers_gdf = station_buffers_gdf.to_crs(epsg=4326)
    station_loc_gdf = station_loc_gdf.to_crs(epsg=4326)
    grid_cells_gdf = grid_cells_gdf.to_crs(epsg=4326)
    centroids_gdf = centroids_gdf.to_crs(epsg=4326)
    road_points_gdf = road_points_gdf.to_crs(epsg=4326)
    village_points_gdf = village_points_gdf.to_crs(epsg=4326)
    
    # Filter buffers for 25km only
    station_buffers_gdf = station_buffers_gdf[station_buffers_gdf['buffer_km'] == 25]
    
    # Join grid_cells data to centroids to get nearest_road_maps_link
    centroids_gdf = centroids_gdf.merge(grid_cells_gdf[['station_name', 'grid_id', 'nearest_road_maps_link']], on=['station_name', 'grid_id'], how='left')
    
    enum_layers = {
        'grid_cells': grid_cells_gdf,
        'centroids': centroids_gdf,
        'road_points': road_points_gdf,
        'village_points': village_points_gdf
    }
    
    return station_buffers_gdf, station_loc_gdf, enum_layers
# Map 1: Aisa FM, Dwanwana FM, and Dokolo FM
station_buffers_gdf, station_loc_gdf, enum_layers = load_all_geodata(PROCESSED_DATA_DIR)

#export the grid_cells. 
enum_layers['grid_cells'].to_csv(PROCESSED_DATA_DIR/'sampling_clusters_full_data.csv')




In [3]:
print(enum_layers['grid_cells'].columns)
print(enum_layers['centroids'].columns)
print(enum_layers['road_points'].columns)
print(enum_layers['village_points'].columns)



#enum_layers['grid_cells']['station_name'].value_counts(dropna=False)

#enum_layers.keys()

Index(['grid_id', 'station_name', 'buffer_km', 'est_population_2020',
       'centroid_lat', 'centroid_lon', 'centroid_maps_link', 'cluster_type',
       'nearest_road_lat', 'nearest_road_lon', 'distance_to_road',
       'nearest_road_maps_link', 'found_at_radius', 'processing_success',
       'nearest_address_full', 'village', 'district', 'region', 'country',
       'nearest_village_lat', 'nearest_village_lon',
       'dist_point_to_nearest_address_km', 'geometry'],
      dtype='object')
Index(['grid_id', 'station_name', 'cluster_type', 'centroid_lat',
       'centroid_lon', 'centroid_maps_link', 'geometry',
       'nearest_road_maps_link'],
      dtype='object')
Index(['grid_id', 'station_name', 'nearest_road_lat', 'nearest_road_lon',
       'distance_to_road', 'nearest_road_maps_link', 'geometry'],
      dtype='object')
Index(['grid_id', 'station_name', 'nearest_address_full', 'village',
       'district', 'region', 'country', 'nearest_village_lat',
       'nearest_village_lon', 'di

In [7]:

def create_station_map(station_names: list, 
                      station_loc_gdf: gpd.GeoDataFrame,
                      enum_layers: dict,
                      zoom_start: int = 10):
    # Filter data for specified stations
    station_locs = station_loc_gdf[station_loc_gdf['station_name'].isin(station_names)]
    
    # Create base map
    m = folium.Map(
        location=[station_locs.geometry.y.mean(), station_locs.geometry.x.mean()],
        zoom_start=zoom_start,
        tiles=None  # Start with no base map
    )
    
    # Add different tile layers
    folium.TileLayer(
        tiles='openstreetmap',
        name='OpenStreetMap'
    ).add_to(m)
    
    folium.TileLayer(
        tiles='https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}',
        attr='Esri',
        name='Satellite',
        overlay=False
    ).add_to(m)

    # Create legend
    legend_html = """
    <div style="position: fixed; 
                bottom: 50px; right: 50px; width: 200px; height: auto;
                background-color: white;
                border: 2px solid grey;
                z-index: 1000;
                padding: 10px;
                font-size: 14px;">
    <p><strong>Radio Stations</strong></p>
    """
    
    for idx, row in station_locs.iterrows():
        legend_html += f"""
        <p><i class="fa fa-circle" style="color:{row['color']}"></i> {row['station_name']}</p>
        """
    
    legend_html += """
    <p><strong>Grid Cells</strong></p>
    <p>● Solid - Main<br>● Dashed - Replacement</p>
    """
    
    if not enum_layers['village_points'].empty and not enum_layers['village_points'].geometry.isna().all():
        legend_html += "<p>● Purple - Nearest Village</p>"
    
    legend_html += """
    </div>
    """
    
    m.get_root().html.add_child(folium.Element(legend_html))
    
    # Create feature groups for each station
    station_groups = {}
    for station in station_names:
        station_groups[station] = folium.FeatureGroup(name=f'{station}')
        
        # Get station color
        station_color = station_locs[station_locs['station_name'] == station]['color'].iloc[0]
        
        # Add grid cells (use different styles for main and replacement)
        station_grids = enum_layers['grid_cells'][enum_layers['grid_cells']['station_name'] == station]
        grid_layer = folium.GeoJson(
            station_grids,
            name=f"{station}_grids",
            style_function=lambda feature: {
                'fillColor': None,
                'color': station_color,
                'weight': 2,
                'fillOpacity': 0,
                'dashArray': '5,5' if feature['properties']['cluster_type'] == 'replacement' else None
            }
        )
        grid_layer.add_to(station_groups[station])
        
        # Add centroid markers
        station_centroids = enum_layers['centroids'][enum_layers['centroids']['station_name'] == station]
        for idx, row in station_centroids.iterrows():
            if row.geometry is not None:
                folium.CircleMarker(
                    location=[row.geometry.y, row.geometry.x],
                    radius=4,
                    color=station_color,
                    fill=True,
                    weight=2,
                    tooltip=folium.Tooltip(f"""
                        Grid ID: {row.grid_id}
                        Station: {row.station_name}
                        Cluster Type: {row['cluster_type']}
                        Nearest Road: <a href='{row.nearest_road_maps_link}' target='_blank'>Link</a>
                        Centroid: <a href='{row.centroid_maps_link}' target='_blank'>Link</a>
                    """),
                    popup=folium.Popup(
                        f"""<div style='width: 300px'>
                        <b style="color:{station_color}">Grid ID:</b> {row.grid_id}<br>
                        <b style="color:{station_color}">Station:</b> {row.station_name}<br>
                        <b style="color:{station_color}">Cluster Type:</b> {row['cluster_type']}<br>
                        <a href='{row.nearest_road_maps_link}' target='_blank'>Nearest Road</a><br>
                        <a href='{row.centroid_maps_link}' target='_blank'>Centroid</a>
                        </div>""",
                        max_width=300
                    )
                ).add_to(station_groups[station])
        
        # Add village point markers
        station_villages = enum_layers['village_points'][enum_layers['village_points']['station_name'] == station]
        for idx, row in station_villages.iterrows():
            if row.geometry is not None and not pd.isna(row.geometry):
                try:
                    folium.CircleMarker(
                        location=[row.geometry.y, row.geometry.x],
                        radius=4,
                        color='purple',
                        fill=True,
                        weight=2,
                        tooltip=folium.Tooltip(f"""
                            Nearest address found in grid cell:
                            {row.nearest_address_full}
                            Village: {row.village}
                            District: {row.district}
                            Region: {row.region}
                        """),
                        popup=folium.Popup(
                            f"""<div style='width: 300px'>
                            <b>Grid ID:</b> {row.grid_id}<br>
                            <b>Nearest Address:</b> {row.nearest_address_full}<br>
                            <b>Village:</b> {row.village}<br>
                            <b>District:</b> {row.district}<br>
                            <b>Region:</b> {row.region}
                            </div>""",
                            max_width=300
                        )
                    ).add_to(station_groups[station])
                except:
                    continue
        
        # Add station marker
        station_loc = station_locs[station_locs['station_name'] == station].iloc[0]
        folium.Marker(
            location=[station_loc.geometry.y, station_loc.geometry.x],
            popup=f"<span style='color:{station_color};'>{station_loc['station_name']}</span>",
            tooltip=folium.Tooltip(f"Click to see {station_loc['station_name']} location"),
            icon=folium.Icon(
                color='white',
                icon_color=station_color,
                icon='radio',
                prefix='fa'
            )
        ).add_to(station_groups[station])
    
    # Add all groups to the map
    for group in station_groups.values():
        group.add_to(m)
    
    # Add layer control
    folium.LayerControl(collapsed=False).add_to(m)
    
    # Save the map to an HTML file
    m.save('radio_station_map.html')

    return m


# Map 1: Aisa FM, Dwanwana FM, and Dokolo FM
station_buffers_gdf, station_loc_gdf, enum_layers = load_all_geodata(PROCESSED_DATA_DIR)
map1 = create_station_map(
    ['Aisa FM', 'Dwanwana FM', 'Dokolo FM'],
    station_loc_gdf,
    enum_layers
)
map1

KeyError: 'cluster_type'

In [5]:
enum_layers['grid_cells'].columns

Index(['grid_id', 'station_name', 'buffer_km', 'est_population_2020',
       'centroid_lat', 'centroid_lon', 'centroid_maps_link', 'cluster_type',
       'nearest_road_lat', 'nearest_road_lon', 'distance_to_road',
       'nearest_road_maps_link', 'found_at_radius', 'processing_success',
       'nearest_address_full', 'village', 'district', 'region', 'country',
       'nearest_village_lat', 'nearest_village_lon',
       'dist_point_to_nearest_address_km', 'geometry'],
      dtype='object')

In [6]:
enum_layers['centroids'].columns

Index(['grid_id', 'station_name', 'cluster_type', 'centroid_lat',
       'centroid_lon', 'centroid_maps_link', 'geometry',
       'nearest_road_maps_link'],
      dtype='object')